In [48]:
!pip install pandas geopy openpyxl
!pip install xlrd



In [50]:
import pandas as pd
from geopy.distance import geodesic

# Load the Gen Rent locations file with lat and lon
gen_rent_df = pd.read_excel("gen rents with lat long.xlsx")

# Check Gen Rent columns
print("Gen Rent Columns:", gen_rent_df.columns)

# Load the area codes file without headers and assign correct column names
area_codes_df = pd.read_csv("us-area-code-cities.xls", header=None)
area_codes_df.columns = ['Area Code', 'City', 'State', 'Country', 'Latitude', 'Longitude']

# Verify column names
print("Area Codes Columns:", area_codes_df.columns)
print(area_codes_df.head())



Gen Rent Columns: Index(['Branch Name', 'Branch Code', 'Address', 'City', 'State', 'Zip Code',
       'Phone', 'After Hours Phone', 'Latitude', 'Longitude'],
      dtype='object')
Area Codes Columns: Index(['Area Code', 'City', 'State', 'Country', 'Latitude', 'Longitude'], dtype='object')
   Area Code            City       State Country  Latitude  Longitude
0        201         Bayonne  New Jersey      US  40.66871  -74.11431
1        201     Bergenfield  New Jersey      US  40.92760  -73.99736
2        201  Cliffside Park  New Jersey      US  40.82149  -73.98764
3        201       Englewood  New Jersey      US  40.89288  -73.97264
4        201       Fair Lawn  New Jersey      US  40.94038  -74.13181


In [54]:
# Rename columns if necessary for consistency
# For example, if 'Area Code' is actually named 'area_code' or 'Code' in your dataset:
# area_codes_df.rename(columns={'Code': 'Area Code'}, inplace=True)

# Add a column for formatted area codes with '+1'
# Adjust 'Area Code' to match the exact column name from Step 2
area_codes_df['Formatted Area Code'] = '+1' + area_codes_df['Area Code'].astype(str)


In [59]:
# Create a list to store the nearest Gen Rent location for each area code
nearest_branch_data = []

# Loop through each area code and find the nearest Gen Rent location
for _, area_row in area_codes_df.iterrows():
    area_lat = area_row['Latitude']
    area_lon = area_row['Longitude']
    area_code = area_row['Formatted Area Code']

    # Find the nearest Gen Rent branch
    min_distance = float('inf')
    nearest_branch = None

    for _, branch_row in gen_rent_df.iterrows():
        branch_lat = branch_row['Latitude']
        branch_lon = branch_row['Longitude']
        branch_name = branch_row['Branch Name']
        
        # Calculate the distance between the area code and the Gen Rent branch
        distance = geodesic((area_lat, area_lon), (branch_lat, branch_lon)).miles
        
        # Check if this is the closest branch
        if distance < min_distance:
            min_distance = distance
            nearest_branch = branch_name

    # Append the result to the list
    nearest_branch_data.append({
        'Area Code': area_code,
        'Nearest Branch': nearest_branch
    })



In [58]:
# Create a DataFrame from the results
nearest_branch_df = pd.DataFrame(nearest_branch_data)

# Save the results to a CSV file
nearest_branch_df.to_csv('gen_rents_lat_long_updated.csv', index=False)

print("CSV file 'gen_rents_lat_long_updated.csv' has been created successfully.")



CSV file 'gen_rents_lat_long_updated.csv' has been created successfully.


In [62]:
# Remove duplicate area codes
nearest_branch_df_cleaned = nearest_branch_df.drop_duplicates(subset='Area Code')

# Save the cleaned results to a CSV file
nearest_branch_df_cleaned.to_csv('gen_rents_lat_long_cleaned.csv', index=False)

print("Cleaned CSV file 'gen_rents_lat_long_cleaned.csv' has been created successfully.")


Cleaned CSV file 'gen_rents_lat_long_cleaned.csv' has been created successfully.
